<a href="https://colab.research.google.com/github/jansoe/KISchule/blob/main/A7_3_jan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 7. Transformer

## 7.3 Nachtrainieren von GPT-2

GPT-2 wurde auf einem Korpus von Wikipedia und Reddit-Texten trainiert. Dementsprechend folgen die generierten Texte dem Duktus dieser Dokumente.

In diesem Notebook wollen wir nun explorieren, wie gut das Finetuning (Nachtrainieren) dieses Modell klappt, wenn wir hierfür eine andere Textdomäne verwenden: Gedichte.

Dieses Notebook orientiert sich dabei stark an diesem Blog: https://www.gwern.net/GPT-2 

### 7.3.0 Vorbereitungen

In [ ]:
%tensorflow_version 2
!pip install transformers==2.4
import transformers
import torch.utils.data as datatools
import torch
import gzip, json
from pprint import pprint
import os

In [ ]:
torch.cuda.get_device_name()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Analog zu Abschnitt 7.0.0 wählen Sie hier Ihren Ordner aus, in dem das nachtrainierte Modell abgespeichert werden soll.

In [ ]:
# Hier werden wir Daten speicher und laden
base_dir = '/content/drive/My Drive/KI-Schule/2 - Textdaten' # dieser Ordner muss in Ihrem Google Drive bereits existieren
new_dir = '/GPT-2'
model_dir = base_dir + new_dir

In [ ]:
!mkdir '{model_dir}'

In [ ]:
!ls '{model_dir}'

### 7.3.1 Das Modell

In [ ]:
model_name_or_path = 'gpt2' # vor dem Nachtrainieren
# model_name_or_path = model_dir + '2'  # Letzte Epoche des nachtrainierten Modells

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name_or_path)
model = transformers.AutoModelWithLMHead.from_pretrained(model_name_or_path)
model.to('cuda')

# Jetzt leiten wir aus den Modell Pfad die letzte Trainingsepoche ab
# oder setzen diese auf -1 wenn wir mit dem Modelltraining von vorne beginnen
last_epoch = -1 if 'drive' not in model_name_or_path else int(model_name_or_path.split('/')[-1])

Jetzt können wir schauen, was das Modell mit einem poetischeren `prompt_text` anfängt:

In [ ]:
prompt_text = ''' 
Born out of numbers
forced to rhyme!
Why
'''

encoded_prompt = tokenizer.encode(prompt_text, return_tensors="pt")
encoded_prompt = encoded_prompt.to('cuda')

output_sequences = model.generate(
    input_ids=encoded_prompt,
    max_length=48,
    do_sample=True,
    temperature=2,
    top_p=0.5,
    repetition_penalty=2
)

generated_sequence = output_sequences[0].tolist()
text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

print(text)

Das Ergebnis sieht eher nicht nach einem Gedicht aus. Deswegen wollen wir versuchen, dem Model mit einem Gedichtkorpus etwas Sprachpoesi zu vermitteln.

### 7.3.2 Der Trainingsdatensatz

Wir laden eine kuratierte Liste von frei verfügbaren Gedichten herunter (https://github.com/aparrish/gutenberg-poetry-corpus) und erstellen aus dieser einen Trainingsdatensatz.

In [ ]:
!curl -O http://static.decontextualize.com/gutenberg-poetry-v001.ndjson.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 52.2M  100 52.2M    0     0  58.0M      0 --:--:-- --:--:-- --:--:-- 58.0M


In [ ]:
#@title
#@markdown Bitte ausführen: Funktion zum Konvertierten des Downloads in einfachen Text.
def convert_to_plain_txt(infile, outfile):
    prev_doc_id = None
    with open(outfile, 'w') as outfile:
        all_lines = []
        for line in gzip.open(infile):
            data = json.loads(line.strip())
            doc_id = data['gid']
            if doc_id != prev_doc_id:
                outfile.write('\n\n')
            text = data['s']
            outfile.write(text + '\n')
            prev_doc_id = doc_id

In [ ]:
# Anwendung der zuvor definierten Funktion
convert_to_plain_txt(infile = "gutenberg-poetry-v001.ndjson.gz", outfile = 'poems.txt')

In [ ]:
#@title
#@markdown Bitte ausführen: Definition einer Dataset-Klasse, die Text als Token bereitstellt.

class TextDataset(datatools.Dataset):

    def __init__(self, tokenizer, file_path, block_size):
        directory, filename = os.path.split(file_path)

        self.examples = []

        # Öffnen der Textdatei
        with open(file_path, encoding="utf-8") as f:
            text = f.read()

        # Umwandeln in numerierte Token
        tokenized_text = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))
        # Zerlegen in Fragmente der Länge block_size
        for i in range(0, len(tokenized_text) - block_size + 1, block_size):  
            example = tokenizer.build_inputs_with_special_tokens(tokenized_text[i : i + block_size])
            self.examples.append(example)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item])

In [ ]:
# Entsprechend auch hier die eigentliche Anwendung der Dataset-Klasse
dataset = TextDataset(
    tokenizer,   
    file_path = 'poems.txt',
    block_size = 64  # Länge der Textfragmente, die wir lernen zu vervollständigen
)

### 7.3.3 Das Training

In [ ]:
optimizer = transformers.AdamW(model.parameters(), lr=5e-4, eps=1e-8)

Wir teilen unseren Datensatz in einen Trainings- und Testdatensatz. Der Fehler auf dem Testdatensatz zeigt uns an, wie gut wir Gedichtzeilen, die nicht im Training vorgekommen sind, vervollständigen können.

In [ ]:
train_size = int(0.95 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [ ]:
# Ein Dataloader für die Trainingsdaten
train_dataloader = datatools.DataLoader(
    train_dataset, 
    shuffle=True, 
    batch_size=32, 
    collate_fn=lambda x: torch.nn.utils.rnn.pad_sequence(x, batch_first=True)
)
len(train_dataloader)

In [ ]:
# Ein Dataloader für die Trainingsdaten
test_dataloader = datatools.DataLoader(
    test_dataset, 
    batch_size=32, 
    collate_fn=lambda x: torch.nn.utils.rnn.pad_sequence(x, batch_first=True)
)
len(test_dataloader)

In [ ]:
#@title
#@markdown Bitte ausführen: Definition der Funktion evaluate() zum Berechnen des Testfehlers.

def evaluate(dataloader, model):
    model.eval()
    eval_loss = 0 
    for step, batch in enumerate(dataloader):
        inputs = batch.to('cuda')

        with torch.no_grad():
            outputs = model(inputs, labels=inputs)
            lm_loss = outputs[0]
            eval_loss += lm_loss.item()

    eval_loss = eval_loss / (step+1)

    print(f'Validation loss {eval_loss}')

Die Trainingsschleife ähnelt der aus Abschnitt 7.1.

In [ ]:
NUM_TRAIN_EPOCHS = 3

GRADIENT_ACCUMULATION_STEPS = 1
MAX_GRAD_NORM = 1.0

for epoch in range(last_epoch+1, last_epoch+1+NUM_TRAIN_EPOCHS):
    print(f"== EPOCH {epoch} started ==" )
    train_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):

        inputs = batch.to('cuda')
        outputs = model(inputs, labels=inputs)
        loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

        train_loss += loss.item()

        if GRADIENT_ACCUMULATION_STEPS > 1:
            loss = loss / GRADIENT_ACCUMULATION_STEPS

        loss.backward()
        
        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)
            optimizer.step()
            model.zero_grad()

        if (step+1) % 1000 == 0:
            print(f'Train Loss {train_loss/(1000):.3f}')
            train_loss = 0


    last_epoch = epoch
    # Speichern des (Zwischen)-Modell
    save_dir = os.path.join(model_dir, str(epoch))
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    model.save_pretrained(save_dir)
    tokenizer.save_pretrained(save_dir)
    torch.save(optimizer.state_dict(), os.path.join(save_dir, "optimizer.pt"))

    evaluate(test_dataloader, model)


### 7.3.4 Evaluierung des nachtrainierten Modells

In [ ]:
prompt_text = ''' 
Born out of numbers
trained by you.
Why
'''

encoded_prompt = tokenizer.encode(prompt_text, return_tensors="pt")
encoded_prompt = encoded_prompt.to('cuda')

output_sequences = model.generate(
    input_ids=encoded_prompt,
    max_length=48,
    do_sample=True,
    temperature=5,
    top_p=0.5,
    repetition_penalty=3
)

generated_sequence = output_sequences[0].tolist()
text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

print(text)

![insitubytes](https://drive.google.com/uc?id=1EAJK7AI9tcZRo3VvYq7vEKGxk7vmK2Ff)